Extracting chapters 

In [1]:
# this is working but need to annotate. 
import os

def extract_chapters(book_file):
    with open(book_file, 'r', encoding='utf-8') as f:
        book_text = f.read()

    chapters = book_text.split("CHAPTER")
    chapters = chapters[1:]

    # Adjusting the output directory to a relative path
    output_dir = "../data/extracted_chapters"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    for i, chapter_text in enumerate(chapters, start=1):
        chapter_title = f"Chapter_{i}"
        chapter_filename = f"{output_dir}/{chapter_title}.txt"

        with open(chapter_filename, 'w', encoding='utf-8') as chapter_file:
            chapter_file.write(chapter_text.strip())

        print(f"Chapter {i} extracted and saved as '{chapter_filename}'.")

# Adjusting the input file to a relative path
extract_chapters("../data/dahl-boy-4.txt")


Chapter 1 extracted and saved as '../data/extracted_chapters/Chapter_1.txt'.
Chapter 2 extracted and saved as '../data/extracted_chapters/Chapter_2.txt'.
Chapter 3 extracted and saved as '../data/extracted_chapters/Chapter_3.txt'.
Chapter 4 extracted and saved as '../data/extracted_chapters/Chapter_4.txt'.
Chapter 5 extracted and saved as '../data/extracted_chapters/Chapter_5.txt'.
Chapter 6 extracted and saved as '../data/extracted_chapters/Chapter_6.txt'.
Chapter 7 extracted and saved as '../data/extracted_chapters/Chapter_7.txt'.
Chapter 8 extracted and saved as '../data/extracted_chapters/Chapter_8.txt'.
Chapter 9 extracted and saved as '../data/extracted_chapters/Chapter_9.txt'.
Chapter 10 extracted and saved as '../data/extracted_chapters/Chapter_10.txt'.
Chapter 11 extracted and saved as '../data/extracted_chapters/Chapter_11.txt'.
Chapter 12 extracted and saved as '../data/extracted_chapters/Chapter_12.txt'.
Chapter 13 extracted and saved as '../data/extracted_chapters/Chapter_

Running the booknlp pipeline


In [5]:
# this is not working for me 

from booknlp.booknlp import BookNLP

model_params = {
    "pipeline": "entity",
    "model": "big"
}

booknlp = BookNLP("en", model_params)

# Relative path to the Chapter_1.txt file inside the data directory
input_file = "../data/extracted_chapters/Chapter_1.txt"

# Relative path to the boy_results directory
# Assuming you want to create this directory at the same level as data and scripts
output_dir = "../boy_results"

book_id = "dahl_boy2"

booknlp.process(input_file, output_dir, book_id)



{'pipeline': 'entity', 'model': 'big'}


HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'C:\Users\Raquel Coelho\booknlps\entities_google/bert_uncased_L-6_H-768_A-12'.

Analyze files
BTW, you can use markdown notation in these cells! Add # for first heading, ## for second heading, etc. It will also allow you to collapse cells under a heading.

#My analyses


#Emotion Detection

In [ ]:
# Raquel - I have not look at this yet
!pip install transformers
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
import json

quotes_file = "/content/drive/MyDrive/Colab_Notebooks/boy_results/dahl_boy.quotes"

def get_descriptors(file):
  main_ID = {472:"Mother", 0:"Boy", 679:"Headmaster", 172:"Coombes", 174:"Hardcastle"}
  classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)

  quotes = pd.read_csv(file, sep = '\t')

  emotions = {"Mother":{"anger": 0, "disgust":0, "fear":0, "joy":0, "sadness":0, "surprise":0}
              ,"Boy":{"anger": 0, "disgust":0, "fear":0, "joy":0, "sadness":0, "surprise":0}
              ,"Headmaster":{"anger": 0, "disgust":0, "fear":0, "joy":0, "sadness":0, "surprise":0},
              "Coombes":{"anger": 0, "disgust":0, "fear":0, "joy":0, "sadness":0, "surprise":0},
              "Hardcastle":{"anger": 0, "disgust":0, "fear":0, "joy":0, "sadness":0, "surprise":0}}



  for index, row in quotes.iterrows():
    if row["char_id"] in main_ID:
      name = main_ID[row["char_id"]]
      temp = classifier(row["quote"])
      for i in range(7):
        if i == 4:
          continue
        emotion = temp[0][i]["label"]
        value = temp[0][i]["score"]
        if value > .3:
          emotions[name][emotion] += 1

  data = json.dumps(emotions)
  json_file = "/content/drive/Shareddrives/ELALytics/Visualizations/Boy/Data/emotions_by_chararcter.json"
  with open(json_file, 'w') as file:
    file.write(data)



      # anger.append(temp[0][0]["score"])
      # disgust.append(temp[0][1]["score"])
      # fear.append(temp[0][2]["score"])
      # joy.append(temp[0][3]["score"])
      # neutral.append(temp[0][4]["score"])
      # sadness.append(temp[0][5]["score"])
      # surprise.append(temp[0][6]["score"])

  # # plt.hist(anger)
  # plt.hist(sadness)
  # # plt.hist(neutral)
  # # plt.hist(joy)

    #use .4 as the marker
get_descriptors(quotes_file)






#Sentiment Analysis

In [ ]:
import spacy
!pip install textblob

from textblob import TextBlob

nlp = spacy.load("en_core_web_sm")

dir = "/content/drive/MyDrive/Colab_Notebooks/boy_results/extracted_chapters/"

def sentences(input):
  sentiment_scores = []
  with open(input, "r", encoding="utf-8") as f:
    text = f.read()

  doc = nlp(text)
  for sent in doc.sents:
    sentiment = TextBlob(sent.text).sentiment.polarity
    sentiment_scores.append(sentiment)

  average = sum(sentiment_scores) / len(sentiment_scores)
  return average

i = 1
dict = {}
for file in os.listdir(dir):
  f = os.path.join(dir, file)
  avg = sentences(f)
  temp = "Chapter_" + str(i)
  dict[temp] = avg
  print(f"Chapter {i} done")
  i += 1

data = json.dumps(dict)
json_file = "/content/drive/Shareddrives/ELALytics/Visualizations/Boy/Data/chapter_sentiment.json"
with open(json_file, 'w') as file:
  file.write(data)



#Conflict

In [ ]:
import pandas as pd
import spacy
import json

nlp = spacy.load("en_core_web_sm")
lexicon_df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/expanding_ela/boy_results/NRC-VAD-Lexicon.csv", sep = "\t")


text = ""

with open("/content/drive/MyDrive/Colab_Notebooks/Copy of 0. The Short Story of _The Lottery_ by_ Shirley Jackson.txt", "r") as file:
  text = file.read()

paragraphs = []

for i in range(1, 82):
  look = f"P{i}"
  look2 = f"P{i + 1}"
  start = text.find(look)
  end = text.find(look2)
  if i == 81:
    para = text[start + 3:]
  else:
    if i > 9:
      para = text[start + 3:end]
    else:
      para = text[start+2:end]
  paragraphs.append(para)

for para in paragraphs:
  print(para)


def get_negative_valence_arousal_score(sentence, lexicon_df):
    # Tokenize sentence into words
    doc = nlp(sentence)

    # Look up valence and arousal scores for each word in the lexicon
    scores = []

    for token in doc:
      word = token.text
      if not word.isalnum():
        continue
      word_data = lexicon_df.loc[lexicon_df['Word'] == word]
      if not word_data.empty:
          valence = word_data['Valence'].values[0]
          arousal = word_data['Arousal'].values[0]
          # Compute negative valence (unpleasantness) by subtracting valence score from 1
          negative_valence = 1 - valence
          # Compute negative valence + arousal
          scores.append(negative_valence + arousal)

  # Calculate average score at the sentence level
    if scores:
        return sum(scores) / len(scores)
    else:
        return None

para_scores = {}
for index, para in enumerate(paragraphs):
  sentence_scores = []
  doc = nlp(para)
  max_sent = ""
  max_score = -100
  for sent in doc.sents:
    score = get_negative_valence_arousal_score(sent.text, lexicon_df)
    if score and score > max_score:
      max = score
      max_sent = sent.text
    sentence_scores.append(score)
  sentence_scores = [score for score in sentence_scores if score is not None]
  if not sentence_scores:
    print(para)
    continue
  average_score = sum(sentence_scores) / len(sentence_scores)
  para_scores[index + 1] = [average_score , max_sent]

data = json.dumps(para_scores)

with open("/content/drive/Shareddrives/ELALytics/Visualizations/Boy/Data/letter_conflict_with_sentences.json", "w") as json_file:
  json_file.write(data)


#word cloud

In [ ]:
#THIS IS THE DATA FOR WORD CLOUD. Extracts most common lemmas
#doesn't filter out filler words / articles like 'and' or 'uh', in
#future I might add a filter to this

# !pip install stop_words
# !pip install nltk

# !pip install booknlp

# from booknlp.booknlp import BookNLP
# import os


# #sets the pipeline, we want to do a full pipeline (usually)
# model_params={
#                 "pipeline":"entity,event,coref,quote,supersense",
#                 "model":"big"

#                 }

# booknlp=BookNLP("en", model_params)


input_dir = "/content/drive/MyDrive/Colab_Notebooks/boy_results/extracted_chapters/"
output_dir = "/content/drive/MyDrive/Colab_Notebooks/boy_results/extracted_chapters/"

####################

import nltk
import json
import pandas as pd

punc = ["!", " ", ".", ",", "?", "-", "|", "'s", ";", "'", "...", "(", ")"]

nltk.download('stopwords')
from nltk.corpus import stopwords
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

# token_file = 'drive/MyDrive/Colab_Notebooks/boy_results/tokens.tsv'
# path = 'drive/MyDrive/Colab_Notebooks/boy_results/'

# output_name = "topwordsPOS"
def get_tokens(token_file, path, name):

  token_df = pd.read_csv(token_file, sep = '\t')
  lemmas_POS = token_df[["lemma","POS_tag"]]
  words = lemmas_POS["lemma"]

  def filter(word):
    if word not in stopwords.words('english') and not word in punc:
      return True
    return False

  filtered = words[words.apply(filter)]
  counts = filtered.value_counts().reset_index()
  counts = counts[:100]
  counts.columns = ["lemma", "Counts"]
  lemmas_POS = pd.merge(lemmas_POS, counts, on="lemma", how = "left")
  lemmas_POS = lemmas_POS.dropna(subset="Counts")
  lemmas_POS.drop_duplicates(subset=["lemma"],inplace=True)
  lemmas_POS = lemmas_POS.sort_values(by="Counts", ascending = False)
  lemmas_POS.rename(columns={"lemma":"word", "POS_tag":"category", "Counts":"value"}, inplace=True)
  lemmas_POS.to_csv(path + name + ".tsv", sep = '\t')
  dict = lemmas_POS.to_dict(orient='records')
  data = json.dumps(dict)
  json_file = path + name + ".json"
  with open(json_file, 'w') as file:
    file.write(data)

# get_tokens(token_file)

i = 1
for file in os.listdir(input_dir):
  if not file.endswith(".tokens"):
    continue
  f = os.path.join(input_dir, file)
  name = "boy_topwords_chapter_" + str(i)
  get_tokens(f, output_dir, name)
  i += 1
  # book_id = "dahl_boy_chapter_" + str(i)
  # booknlp.process(f, output_dir, book_id)
  # i += 1










[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Distinctive Words


In [ ]:
#create words df

import pandas as pd
import os
import nltk
punc = ["!", " ", ".", ",", "_", "?", "-", "|", "'s", ";", "'", "...", "(", ")"]

nltk.download('stopwords')

from nltk.corpus import stopwords

chapter_names = ["Papa and Mama", "Kindergarten, 1922-3 (age 6-7)", "The bicycle and the sweet-shop",
                 "The Great Mouse Plot", "Mr Coombes", "Mrs Pratchett's Revenge", "Going to Norway",
                 "The Magic Island", "A Visit to the Doctor", "First Day", "Writing Home", "The Matron",
                 "Homesickness", "A Drive in the Motor-car", "Captain Hardcastle", "Little Ellis and the Boil",
                 "Goat's Tobacco", "Getting Dressed for the Big School", "Boazers", "The Headmaster",
                 "Chocolates","Corkers", "Being Bossed Around", "Games and Photography",
                 "Goodbye School"]

input_dir = "/content/drive/MyDrive/Colab_Notebooks/boy_results/extracted_chapters"

from wordcloud import WordCloud
import matplotlib.pyplot as plt

def get_tokens(token_file, chapter):

  token_df = pd.read_csv(token_file, sep = '\t')
  lemmas_POS = token_df[["lemma","POS_tag"]]
  words = lemmas_POS["lemma"]

  def filter(word):
    if word not in stopwords.words('english') and not word in punc:
      return True
    return False

  filtered = words[words.apply(filter)]
  counts = filtered.value_counts().reset_index()
  counts.columns = ["lemma", "Counts"]
  lemmas_POS = pd.merge(lemmas_POS, counts, on="lemma", how = "left")
  lemmas_POS = lemmas_POS.dropna(subset="Counts")
  lemmas_POS.drop_duplicates(subset=["lemma"],inplace=True)
  lemmas_POS = lemmas_POS.sort_values(by="Counts", ascending = False)
  lemmas_POS["chapter"] = chapter
  lemmas_POS.rename(columns={"lemma":"word", "POS_tag":"category", "Counts":"value"}, inplace=True)

  return lemmas_POS

result = pd.DataFrame(columns=["word", "category", "value"])
i = 1
for file in os.listdir(input_dir):
  if not file.endswith(".tokens"):
      continue
  else:
    file = input_dir + "/" + file
    temp = get_tokens(file, i)
  result = pd.concat([result, temp], ignore_index = True)
  i += 1

word_df = result

# # Define dictionary to map original POS tags to EVEN MORE simplified tags
pos_dict_most_simplified = {'NOUN': 'noun',
            'PROPN': 'noun',
                            "PRON":"noun",

            'VERB': 'verb',
            'ADJ': 'adj',
            'ADV': 'adv',
 }

# Define function to simplify POS tags
def most_simplify_pos_tag(tag):
    return pos_dict_most_simplified.get(tag, 'other')

word_df['pos_more_simple'] = word_df['category'].apply(most_simplify_pos_tag)



# result.to_csv("/content/drive/MyDrive/Colab_Notebooks/boy_results/boy_words.tsv", sep = '\t')

from collections import defaultdict
import math
import operator


def log_odds(counts1, counts2, prior, zscore = True):
    # code from Dan Jurafsky
    # note: counts1 will be positive and counts2 will be negative

    sigmasquared = defaultdict(float)
    sigma = defaultdict(float)
    delta = defaultdict(float)

    n1 = sum(counts1.values())
    n2 = sum(counts2.values())

    # since we use the sum of counts from the two groups as a prior, this is equivalent to a simple log odds ratio
    nprior = sum(prior.values())
    for word in prior.keys():
        if prior[word] == 0:
            delta[word] = 0
            continue
        if word not in counts1:
            counts1[word] = 0
        if word not in counts2:
            counts2[word] = 0
        l1 = float(counts1[word] + prior[word]) / (( n1 + nprior ) - (counts1[word] + prior[word]))
        l2 = float(counts2[word] + prior[word]) / (( n2 + nprior ) - (counts2[word] + prior[word]))
        sigmasquared[word] = 1/(float(counts1[word]) + float(prior[word])) + 1/(float(counts2[word]) + float(prior[word]))
        sigma[word] = math.sqrt(sigmasquared[word])
        delta[word] = (math.log(l1) - math.log(l2))
        if zscore:
            delta[word] /= sigma[word]
    return delta

prior_counts = word_df.groupby("word")['value'].sum().to_dict()

from matplotlib.colors import ListedColormap
# Define a color function that maps POS tags to colors
# simple_pos_levels = word_df['pos_more_simple'].unique()
# num_pos_levels = len(simple_pos_levels)
# colors = ListedColormap(plt.cm.Set1(range(num_pos_levels)))
# pos_color_dict = dict(zip(simple_pos_levels, colors(range(num_pos_levels))))
# def color_func(word, font_size, position, orientation, random_state=None, **kwargs):
#     return pos_color_dict[word_df.loc[word_df['word'] == word, 'pos_more_simple'].values[0]]


pos_color_dict = {'noun': (31, 119, 180),  # blue
                  'adj': (242, 223, 58),  # yellow
                  'adv': (0, 102, 0),  # green
                  'verb': (245, 139, 213)}  # pink
default_color = (128, 128, 128)  # grey
def color_func2(word, font_size, position, orientation, random_state=None, **kwargs):
    return pos_color_dict.get(word_df.loc[word_df['word'] == word, 'pos_more_simple'].values[0], default_color)

for i in range(1, 26):
  print("CHAPTER " + str(i))
  chap_counts = word_df[word_df['chapter'] == i].set_index('word')['value'].to_dict()
  other_counts = word_df[word_df['chapter'] != i].groupby("word")['value'].sum().to_dict()
  delta = log_odds(chap_counts, other_counts, prior_counts, True)
  delta_sorted = sorted(delta.items(), key=operator.itemgetter(1))
  chap_words = []
  for k, v in reversed(delta_sorted):
      if v >= 1.5:
          chap_words.append("%s (%.3f)" % (k, v))
  to_plot = {k: v for k, v in reversed(delta_sorted) if v > 0}
  wordcloud = WordCloud(width=500, height=500, background_color='white',
                         max_words=100, color_func=color_func2,
                        min_font_size = 15).generate_from_frequencies(to_plot)
  plt.figure(figsize=(5, 5), dpi=800, facecolor=None)
  plt.title(chapter_names[i-1], size=20)
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad=0)
  temp = "CHAPTER" + str(i) + ".png"
  plt.savefig(input_dir + "/" +  temp)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


FileNotFoundError: ignored

In [ ]:
from collections import defaultdict
import math


def log_odds(counts1, counts2, prior, zscore = True):
    # code from Dan Jurafsky
    # note: counts1 will be positive and counts2 will be negative

    sigmasquared = defaultdict(float)
    sigma = defaultdict(float)
    delta = defaultdict(float)

    n1 = sum(counts1.values())
    n2 = sum(counts2.values())

    # since we use the sum of counts from the two groups as a prior, this is equivalent to a simple log odds ratio
    nprior = sum(prior.values())
    for word in prior.keys():
        if prior[word] == 0:
            delta[word] = 0
            continue
        if word not in counts1:
            counts1[word] = 0
        if word not in counts2:
            counts2[word] = 0
        l1 = float(counts1[word] + prior[word]) / (( n1 + nprior ) - (counts1[word] + prior[word]))
        l2 = float(counts2[word] + prior[word]) / (( n2 + nprior ) - (counts2[word] + prior[word]))
        sigmasquared[word] = 1/(float(counts1[word]) + float(prior[word])) + 1/(float(counts2[word]) + float(prior[word]))
        sigma[word] = math.sqrt(sigmasquared[word])
        delta[word] = (math.log(l1) - math.log(l2))
        if zscore:
            delta[word] /= sigma[word]
    return delta


prior_counts = word_df.groupby("word")['count'].sum().to_dict()

for i in chapters:
    print("CHAPTER %d: %s" % (i, chapter_names[i]))
    chap_counts = word_df[word_df['chapter'] == i].set_index('word')['count'].to_dict()
    other_counts = word_df[word_df['chapter'] != i].groupby("word")['count'].sum().to_dict()
    delta = log_odds(chap_counts, other_counts, prior_counts, True)
    delta_sorted = sorted(delta.items(), key=operator.itemgetter(1))
    chap_words = []
    for k, v in reversed(delta_sorted):
        if v >= 1.5:
            chap_words.append("%s (%.3f)" % (k, v))
    print("\n".join(chap_words))
    to_plot = {k: v for k, v in reversed(delta_sorted) if v > 0}
    print(to_plot)
    print('-------')



#Descriptors by Character  

In [ ]:
import json
import pandas as pd
import csv

from google.colab import drive
drive.mount('/content/drive')

book_file = "/content/drive/MyDrive/Colab_Notebooks/boy_results/dahl_boy.book"
quotes_file = "/content/drive/MyDrive/Colab_Notebooks/boy_results/dahl_boy.quotes"

#Copied from Example Colab
# def get_NRC_lexicon():
#     '''
#     @output:
#     - A chars[name][character]ionary of format {word : score}
#     '''
#     lexicon = '/content/drive/MyDrive/Colab_Notebooks/boy_results/NRC-VAD-Lexicon.txt'
#     val_chars[character] = {}
#     aro_chars[character] = {}
#     dom_chars[character] = {}
#     with open(lexicon, 'r') as infile:
#         reader = csv.chars[name][character]Reader(infile, delimiter="\t")
#         for row in reader:
#             word = row['Word']
#             val_chars[character][word] = float(row['Valence'])
#             aro_chars[character][word] = float(row['Arousal'])
#             dom_chars[character][word] = float(row['Dominance'])
#     return (val_chars[character], aro_chars[character], dom_chars[character])

# # save VAD chars[name][character]ionaries
# val_chars[character], aro_chars[character], dom_chars[character] = get_NRC_lexicon()


#extract descriptors by gender + run VAD

def get_descriptors(file):
  main_ID = {472:"Mother", 0:"Boy", 679:"Headmaster", 172:"Coombes", 174:"Hardcastle"}

  quotes = pd.read_csv(file, sep = '\t')

  for index, row in quotes.iterrows():
    if row["char_id"] in main_ID:



  # with open(file, "r") as file:
  #   book_data = json.load(file)


    # if character["mentions"]["proper"]:
    #   if character["mentions"]["proper"][0]['n'] not in main:
    #     continue
    #   else:
    #     name = character["mentions"]["proper"][0]["n"]
    #     ID = character["id"]
    # elif character["mentions"]["common"]:
    #   if character["mentions"]["common"][0]["n"] not in main:
    #     continue
    #   else:
    #     name = character["mentions"]["common"][0]["n"]
    #     ID = character["id"]
    # else:
    #   if character["mentions"]["pronoun"] and character["mentions"]["pronoun"][0]["n"] in main:
    #     name = "I"
    #   else:
    #     continue
    # chars[name] = {"poss":{}, "agents":{}, "pats":{}}
  #   # else:
  #   #   name = character["mentions"]["common"][0]['n']
  #   # chars[name][character] = {"adjs":[], "agents":[], "pats":[]}
  #   # if character["g"] is None:
  #   #   continue
  #   #take character attributes
  #   for poss in character["poss"]:
  #       # possL.append(poss['w'])
  #     if not poss['w'] in chars[name]["poss"]:
  #       chars[name]["poss"][poss['w']] = 0
  #     chars[name]["poss"][poss['w']] += 1
  #   # for patient in character["patient"]:
  #   #     # patsL.append(patient['w'])
  #   #   if not patient['w'] in chars[name]["pats"]:
  #   #     chars[name]["pats"][patient['w']] = 0
  #   #   chars[name]["pats"][patient['w']] += 1
  #   for agent in character["agent"]:
  #     if not agent['w'] in chars[name]["agents"]:
  #       chars[name]["agents"][agent['w']] = 0
  #     chars[name]["agents"][agent['w']] += 1
  #       # agentL.append(agent['w'])
  #   result[name] =  {"poss":{}, "agents":{}, "pats":{}}
  #   for key, val in sorted(chars[name]["agents"].items(), key= lambda tuple: tuple[1] , reverse=True)[:15]:
  #     result[name]["agents"][key] = val
  #   for key, val in sorted(chars[name]["poss"].items(), key= lambda tuple: tuple[1], reverse=True)[:15]:
  #     result[name]["poss"][key] = val
  #   for key, val in sorted(chars[name]["pats"].items(), key = lambda tuple: tuple[1], reverse=True)[:15]:
  #     result[name]["pats"][key] = val
  # print(result)


  # list = []
  # for character in result:
  #   words_list = []
  #   for word, count in result[character]["poss"].items():
  #     words_list.append((word,count,"poss"))
  #   for word, count in result[character]["agents"].items():
  #     words_list.append((word, count, "agent"))
  #   # for word, count in result[character]["pats"].items():
  #   #   words_list.append((word, count, "patient"))
  #   words_list = sorted(words_list, key = lambda x: x[1], reverse=True)
  #   i = 1
  #   for word in words_list[:15]:
  #     if character == "my mother":
  #       gender = "female"
  #     else:
  #       gender = "male"
  #     list.append({"word":word[0], "character":character, "rank":str(i), "count":word[1], "type":word[2], "gender":gender})
  #     i += 1

  # data = json.dumps(list)
  # json_file = "/content/drive/Shareddrives/ELALytics/Visualizations/Boy/Data/character_words_NO_PATIENT.json"
  # with open(json_file, 'w') as file:
  #   file.write(data)


  # poss_series = pd.Series(possL)
  # pats_series = pd.Series(patsL)
  # agent_series = pd.Series(agentL)

  # poss_series = poss_series.value_counts()[:15]
  # agent_series = agent_series.value_counts()[:15]
  # pats_series = pats_series.value_counts()[:15]

  # print(poss_series.index)
  # print(poss_series.values)

  # results = pd.DataFrame({"possessive":poss_series.index, "poss_count":poss_series.values,
  #         "agents":agent_series.index, "age_count":agent_series.values ,
  #         "patient":pats_series.index, "pat_count":pats_series.values})

  # results.to_csv("/content/drive/Shareddrives/ELALytics/Visualizations/Boy/Data/poss_agent_patient_list.tsv",
  #                sep = '\t')
  # dict = results.to_dict(orient="records")






  # print("POSSESSIVE VERBS")
  # for poss in sorted(chars[name]["poss"],key = lambda word: chars[name]["poss"].get(word), reverse=True)[:10]:
  #   print(poss, chars[name]["poss"][poss])
  #   results[""]
  # print("\n")
  # print("PATIENT VERBS")
  # for pat in sorted(chars[name]["pats"],key = lambda word: chars[name]["pats"].get(word), reverse=True)[:10]:
  #   print(pat, chars[name]["pats"][pat])
  # print("\n")
  # print("AGENT VERBS")
  # for agent in sorted(chars[name]["agents"],key = lambda word: chars[name]["agents"].get(word), reverse=True)[:10]:
  #   print(agent, chars[name]["agents"][agent])

    # for name in character["mentions"]["common"][:10]:
    #   chars[name][character]["names"].append(name['n'])
  # male = pd.Series(male_adjs)
  # female = pd.Series(female_adjs)
  # male = male.drop_duplicates()
  # female = female.drop_duplicates()
  # df = pd.DataFrame()
  # df["Male Descriptors"] = male
  # df["Female Descriptors"] = female
  #m_val, m_arous, m_dom, f_val, f_arous, f_dom = ([] for i in range(6))
  #Get VAD scores for adjectives
  # for adj in female_adjs:
  #   if adj in list(val_chars[character].keys()):
  #     f_val.append(val_chars[character][adj])
  #   else:
  #     f_val.append(999)
  #   # if adj in list(aro_chars[character].keys()):
  #   #   f_arous.append(aro_chars[character][adj])
  #   # else:
  #   #   f_arous.append(999)
  #   # if adj in list(dom_chars[character].keys()):
  #   #   f_dom.append(dom_chars[character][adj])
  #   # else:
  #   #   f_dom.append(999)

  # for adj in male_adjs:
  #   if adj in list(val_chars[character].keys()):
  #     m_val.append(val_chars[character][adj])
  #   else:
  #     m_val.append(999)
    # if adj in list(aro_chars[character].keys()):
    #   m_arous.append(aro_chars[character][adj])
    # else:
    #   m_arous.append(999)
    # if adj in list(dom_chars[character].keys()):
    #   m_dom.append(dom_chars[character][adj])
    # else:
    #   m_dom.append(999)

  # df.insert(1, "MValence", m_val)
  #df.insert(2, "MArousal", m_arous)
  #df.insert(3, "MDominance", m_dom)
  # df.insert(5, "FValence", f_val)
  #df.insert(6, "FArousal", f_arous)
  #df.insert(7, "FDominance", f_dom)

  # df = pd.DataFrame()
  # var = [male, female]
  # for gender in var:
  #   chars[name][character]2 = {}
  #   for key, value2 in gender.items():
  #     series = pd.Series(data=value2, name = key)
  #     series = series.drop_duplicates()
  #     if gender == male:
  #       kwargs = {"m" + key:series}
  #     else:
  #       kwargs = {"f" + key:series}
  #     df = df.assign(**kwargs)



  # path = "/content/drive/MyDrive/Colab_Notebooks/boy_results/"
  # df.to_csv(path + "gender_data.csv")


get_descriptors(quotes_file)


















character verbs

In [ ]:
book_file = "/content/drive/MyDrive/Colab_Notebooks/boy_results/dahl_boy.book"


# def get_NRC_lexicon():
#     '''
#     @output:
#     - A dictionary of format {word : score}
#     '''
#     lexicon = '/content/drive/MyDrive/Colab_Notebooks/boy_results/NRC-VAD-Lexicon.txt'
#     val_dict = {}
#     aro_dict = {}
#     dom_dict = {}
#     with open(lexicon, 'r') as infile:
#         reader = csv.DictReader(infile, delimiter="\t")
#         for row in reader:
#             word = row['Word']
#             val_dict[word] = float(row['Valence'])
#             aro_dict[word] = float(row['Arousal'])
#             dom_dict[word] = float(row['Dominance'])
#     return (val_dict, aro_dict, dom_dict)

# # save VAD dictionaries
# val_dict, aro_dict, dom_dict = get_NRC_lexicon()

import json
from google.colab import files

def get_adjs(book_file):
  with open(book_file, "r") as file:
    book_data = json.load(file)
  char_list = []
  words = []
  seen = set()
  i = 0
  for character in book_data["characters"]:
    if character["mentions"]["proper"]:
      name = character["mentions"]["proper"][0]['n']
    # elif character["mentions"]["common"]:
    #   name = character["mentions"]["common"][0]['n']
    else:
      continue
    char_list.append({"character":name, "words":[]})
    for adj in character["mod"][:10]:
      char_list[i]["words"].append(adj['w'])
      if adj['w'] in seen:
        continue
      seen.add(adj['w'])
      words.append(adj['w'])
    for patient in character["patient"][:10]:
      char_list[i]["words"].append(patient['w'])
      if patient['w'] in seen:
        continue
      seen.add(patient['w'])
      words.append(patient['w'])
    for agent in character["agent"][:10]:
      char_list[i]["words"].append(agent['w'])
      if agent['w'] in seen:
        continue
      seen.add(agent['w'])
      words.append(agent['w'])
      # else:
      #   char_list[name][adj['w']] = "N/A"
    i += 1

  # for char in char_list:
  #   print(char,"\n")
  # for word in words:
  #   print(word,"\n")
  # for k, v in char_list.items():
  #   print(k,v,"\n")
  # print(words)

  chars_data = json.dumps(char_list)
  words_data = json.dumps(words)
  path= "/content/drive/MyDrive/Colab_Notebooks/boy_results"
  chars_file = path + "/char2.json"
  words_file = path + "/words2.json"

  with open(chars_file, 'w') as json_file:
    json_file.write(chars_data)

  with open(words_file, 'w') as json_file:
    json_file.write(words_data)

  files.download(chars_file)
  files.download(words_file)


get_adjs(book_file)




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Places

In [ ]:
import json
import pandas as pd
import csv

from google.colab import drive
drive.mount('/content/drive')

entities_file = "/content/drive/MyDrive/Colab_Notebooks/boy_results/dahl_boy.entities"

def get_places(entities_file):
  df = pd.read_csv(entities_file, delimiter = '\t')
  places = []
  cat = []

  for index, row in df.iterrows():
    if row["cat"] != "PER" and row["prop"] == "PROP":
      places.append(row["text"])
      cat.append(row["cat"])
  result = pd.DataFrame(zip(places, cat))
  result = result.drop_duplicates()
  result.to_csv("/content/drive/MyDrive/Colab_Notebooks/boy_results/places.tsv")

get_places(entities_file)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
